In [2]:
sys.path.append('..')
from config import Config

import json
import numpy as np # Для работы с данными 
import sys
import os

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами

In [3]:
JSON_DATA_DIR = '../../Satellites'

Все тексты true и false объединены в два больших текста;
Датасет - список из двух элементов-строк

In [4]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
textClasses = ['', '']


# открываем файл с id эссе и ответами
with open(JSON_DATA_DIR + '/train/train_standart.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]

    with open(JSON_DATA_DIR + f'/train/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      if elem['answer'] == False:
        textClasses[0] += text
        textClasses[0] += '#'
      else:
        textClasses[1] += text
        textClasses[1] += '#'

In [4]:
print(len(textClasses[0].split('.')))
print(len(textClasses[1].split('.')))

24526
6374


In [5]:
print(len(textClasses[0]))
print(len(textClasses[1]))

print(textClasses[0][-50:])
print(textClasses[1][-50:])

texts_false = textClasses[0].split("#")
texts_true = textClasses[1].split("#")

print(len(texts_false))
print(len(texts_true))

print(texts_false[0])

2587553
654756
вторитет СССР и провести политику десталинизации.#
рии и заключается противоречивость Раскольникова.#
801
201
Считаю, почти все дадут согласие, что в протяжении жизни мы ни один раз встречаем безответственных профессионалов собственного дела. К несчастью, если дело касается медицины, то безалаберное отношение может сыграть злую шуточку. Над неувязкой ответственности анализирует автор текста. Василий Павлович Аксенов думает, что к собственной работе нужно относится серьезно и радиво, в особенности если идет речь о здоровье остальных людей. Свое мнение автор обосновывает на примере ситуации из текста.
Василий Павлович Аксенов думает, что к собственной работе нужно относится серьезно и радиво, в особенности если идет речь о здоровье остальных людей. Свое мнение автор обосновывает на примере ситуации из текста. Василий Павлович противопоставляет 2-ух мед сотрудников различных поколений. Юный спец Александр Зеленин чрезвычайно трепетно подступает к реализации собственных обяз

In [6]:
test_border_false = len(texts_false)//10
test_border_true = len(texts_true)//10

val_border_false = len(texts_false)//20 + test_border_false
val_border_true = len(texts_true)//20 + test_border_true


valText = []
valText.append(' '.join(texts_false[:test_border_false]))
valText.append(' '.join(texts_true[:test_border_true]))

testText = []
testText.append(' '.join(texts_false[test_border_false:val_border_false]))
testText.append(' '.join(texts_true[test_border_true:val_border_true]))

trainText = []
trainText.append(' '.join(texts_false[val_border_false:]))
trainText.append(' '.join(texts_true[val_border_true:]))

In [17]:
print(len(testText[0].split(".")))
print(len(testText[1].split(".")))
print(len(valText[0].split(".")))
print(len(valText[1].split(".")))
print(len(trainText[0].split(".")))
print(len(trainText[1].split(".")))

1276
309
2532
655
20720
5412


Список параметров, которые будут меняться в течение генерации датасетов

In [28]:
numWordsList = [10000, 20000, 30000, "all"]
xLenList = [100, 300, 500]
stepList = [
    [(25, 6), (50, 12), (75, 17)],
    [(100, 25), (150, 35), (250, 60)],
    [(150, 35), (250, 60), (350, 85)]
]
bowList = ["BOW", ""]

In [12]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wIclass in range(nClasses):      # Для каждого текста выборки из последовательности индексов
    if wIclass == 0:
      classesXSamples.append(getSetFromIndexes(wordIndexes[0], xLen, step[0]))
    else:
      classesXSamples.append(getSetFromIndexes(wordIndexes[1], xLen, step[1]))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [8]:
# tokenizer = Tokenizer(num_words=20000,
#     filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
#     lower=True,
#     split=' ',
#     oov_token='unknown',
#     char_level=False)

# morphAnalyzer = pymorphy2.MorphAnalyzer
# nw_tokenizer = morphAnalyzer.iter_known_word_parses(tokenizer)
# print(list(nw_tokenizer))

In [9]:
# trainText[0] = text2Words(trainText[0])
# trainText[1] = text2Words(trainText[1])

# tokenizer = Tokenizer(num_words=10000,
#                 filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
#                 lower=True,
#                 split=' ',
#                 oov_token='unknown',
#                 char_level=False)

# tokenizer.fit_on_texts(trainText) # словарь частотности
# items = list(tokenizer.word_index.items()) # индексы слов

# print("Самые часто встречающиеся слова:")
# print(items[:10])
# print()
# print("Самые редко встречающиеся слова:")
# print(items[-10:])
# print()
# print("Размер словаря:", len(items))

In [ ]:
labels = ['False', 'True']
labelsNum = len(labels)

        
for numWords in numWordsList:
    print("numwords", numWords)
    tokenizer = Tokenizer(num_words=numWords,
        filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
        lower=True,
        split=' ',
        oov_token='unknown',
        char_level=False)
        
    if numWords == "all":
        tokenizer = Tokenizer(
            filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
            lower=True,
            split=' ',
            oov_token='unknown',
            char_level=False)
        
        tokenizer.fit_on_texts(trainText)
        items = list(tokenizer.word_index.items())
        numWords = len(items) - 1
        
    tokenizer.fit_on_texts(trainText)

    trainWordIndexes = tokenizer.texts_to_sequences(trainText)
    valWordIndexes = tokenizer.texts_to_sequences(valText)
    testWordIndexes = tokenizer.texts_to_sequences(testText)

    for xLenIndex in range(len(xLenList)):
        print("xLen", xLenList[xLenIndex])
        for step in stepList[xLenIndex]:
            print("step", step)
            xLen = xLenList[xLenIndex]

            xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
            xValId, yVal = createSetsMultiClasses(valWordIndexes, xLen, step)
            xTestId, yTest = createSetsMultiClasses(testWordIndexes, xLen, xLen)

            for bowState in bowList:
                print("bowState", bowState)
                if bowState:
                    xTrain = tokenizer.sequences_to_matrix(xTrainId.tolist())
                    xVal = tokenizer.sequences_to_matrix(xValId.tolist())
                    xTest  = tokenizer.sequences_to_matrix(xTestId.tolist())
                else:
                    xTrain = xTrainId
                    xVal = xValId
                    xTest = xTestId

                np.savez_compressed(f"{Config.DATASETS_DIR}dataset_numWords_{numWords}_xLen_{xLen}_step_{step}_{bowState}.npz", xTrain=xTrain, yTrain=yTrain, xVal=xVal, yVal=yVal, xTest=xTest, yTest=yTest)
        

In [11]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText

def createSetsMultiClassesTest(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wordIndexes[0], xLen, step))


  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [13]:
labels = ['False', 'True']
labelsNum = len(labels)

        
for numWords in numWordsList:
    print("numwords", numWords)
    tokenizer = Tokenizer(num_words=numWords,
        filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
        lower=True,
        split=' ',
        oov_token='unknown',
        char_level=False)
        
    if numWords == "all":
        tokenizer = Tokenizer(
            filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
            lower=True,
            split=' ',
            oov_token='unknown',
            char_level=False)
        
        tokenizer.fit_on_texts(trainText)
        items = list(tokenizer.word_index.items())
        numWords = len(items) - 1
        
    tokenizer.fit_on_texts(trainText)

    trainWordIndexes = tokenizer.texts_to_sequences(trainText)
    valWordIndexes = tokenizer.texts_to_sequences(valText)

    for xLenIndex in range(len(xLenList)):
        print("xLen", xLenList[xLenIndex])
        for step in stepList[xLenIndex]:
            print("step", step)
            xLen = xLenList[xLenIndex]

            xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
            xValId, yVal = createSetsMultiClasses(valWordIndexes, xLen, step)

            for bowState in bowList:
                print("bowState", bowState)
                if bowState:
                    xTrain = tokenizer.sequences_to_matrix(xTrainId.tolist())
                    xVal = tokenizer.sequences_to_matrix(xValId.tolist())
                else:
                    xTrain = xTrainId
                    xVal = xValId

                np.savez_compressed(f"{Config.DATASETS_DIR}dataset_numWords_{numWords}_xLen_{xLen}_step_{step}_{bowState}.npz", xTrain=xTrain, yTrain=yTrain, xVal=xVal, yVal=yVal, xTest=xTest, yTest=yTest)

NameError: name 'numWordsList' is not defined

In [15]:
   
tokenizer = Tokenizer(
    filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    oov_token='unknown',
    char_level=False)

tokenizer.fit_on_texts(trainText)
# items = list(tokenizer.word_index.items())
numWords = 48131

trainWordIndexes = tokenizer.texts_to_sequences(trainText)
valWordIndexes = tokenizer.texts_to_sequences(valText)
testWordIndexes = tokenizer.texts_to_sequences(testText)

xLen = 300
step = (150, 35)

xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xValId, yVal = createSetsMultiClasses(valWordIndexes, xLen, step)
xTestId, yTest = createSetsMultiClassesTest(testWordIndexes, xLen, xLen)

xTrain = xTrainId
xVal = xValId
xTest  = xTestId
np.savez_compressed(f"{Config.DATASETS_DIR}NEWdataset_numWords_{numWords}_xLen_{xLen}_step_{xLen}_.npz", xTrain=xTrain, yTrain=yTrain, xVal=xVal, yVal=yVal, xTest=xTest, yTest=yTest)
